In [1]:
from ngsolve import *
from ngsolve import Draw, curl, grad
from netgen.occ import *
from netgen.gui import *
from coil_geometry import *

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2405
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 14 thread(s)


2024-12-28 15:14:04.667 Python[7208:40203] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-28 15:14:04.667 Python[7208:40203] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [2]:
nwindings = 6  # Number of windings
wireradius = 0.001  # Radius of the wire
coilradius = 0.01  # Radius of the coil

# Geometrie laden
geo = create_homo_geometry(nwindings, wireradius, coilradius)

In [3]:
Draw(geo)

In [ ]:
mesh = Mesh(
    geo.GenerateMesh(maxh= 1.5* wireradius )
)
mesh.Curve(1) 

In [ ]:
mesh.GetBoundaries()

In [ ]:
# Konstanten
mu = 4 * pi * 1e-7  # Magnetische Feldkonstante (S.2)
sigma_coil = 5.87e7  # Leitfähigkeit Kupfer (S.11)
sigma_air = 0 # Leitfähigkeit Luft
k = 1e-5  # Regularisierung

In [ ]:
# H(curl)-Raum definieren
order = 2
V = HCurl(mesh, order=order-1, nograds=True, dirichlet="outer")
u, v = V.TnT()
gfA = GridFunction(V)

# Bilinearform und Linearform
a = BilinearForm(V, symmetric=True)
a += (1/mu) * InnerProduct(curl(u), curl(v)) * dx
a += k * InnerProduct(u, v) * dx

# Linearform: Stromdichte
j_c = CoefficientFunction((0, 0, 1))  # Beispiel: konstant
f = LinearForm(V)
f += InnerProduct(j_c, v) * dx

# Assemble
pre = Preconditioner(a, "multigrid")
a.Assemble()
f.Assemble()

print(f"Matrix size: {a.mat.height} x {a.mat.width}")
print(f"Vector size: {f.vec.size}")
print(f"Präkonditionierer-Matrixgröße: {pre.mat.height} x {pre.mat.width}")

In [ ]:
# Lösen
with TaskManager():
    inv = CGSolver(a.mat, pre, printrates=True)
    gfA.vec.data = inv * f.vec

# Visualisierung
Draw(curl(gfA), mesh, "MagneticField")
Draw(gfA, mesh, "VectorPotential")